In [ ]:
#https://classic.scraperwiki.com/scrapers/all_party_groups/

In [62]:
!mv appg_cache.sqlite appg_cache27.sqlite

In [63]:
url="https://www.publications.parliament.uk/pa/cm/cmallparty/170502/contents.htm"

In [1]:
#!rm appg_cache.sqlite

In [3]:
import requests
import requests_cache
from lxml import etree
from urllib.parse import urljoin

requests_cache.install_cache('appg_cache', backend='sqlite', expire_after=None)



In [4]:
import pandas as pd

r=requests.get('https://www.publications.parliament.uk/pa/cm/cmallparty/170502/germany.htm')
r.from_cache
dfs=pd.read_html(r.text)


In [5]:
gid='tmp'

## Find Links To Registers

In [14]:
url='http://www.parliament.uk/mps-lords-and-offices/standards-and-financial-interests/parliamentary-commissioner-for-standards/registers-of-interests/register-of-all-party-party-parliamentary-groups/'

from bs4 import BeautifulSoup
import re

def _getLinksFromPage(url, textcrib=None, hrefcrib=True):
    
    page = requests.get(url)

    #The file we have grabbed in this case is a web page - that is, an HTML file
    #We can get the content of the page and parse it
    soup=BeautifulSoup(page.content, "html5lib")
    #BeautifulSoup has a routine - find_all() - that will find all the HTML tags of a particular sort
    #Links are represented in HTML pages in the form <a href="http//example.com/page.html">link text</a>
    #Grab all the <a> (anchor) tags...
    
    if textcrib:
        souplinks=soup.find_all('a', text=re.compile(textcrib))
    elif hrefcrib:
        souplinks=soup.find_all('a', href=re.compile(hrefcrib))
    else:
        souplinks=soup.find_all('a')
    return souplinks

links = _getLinksFromPage(url,textcrib='Registers published')

#get the most recent link
recent = sorted([link.text for link in links])[-1]
link = [link for link in links if link.text == recent][0]

url='http://www.parliament.uk'+link['href']
links = _getLinksFromPage(url,hrefcrib='pa/cm/cmallparty.*htm')
links[0]['href']

'https://publications.parliament.uk/pa/cm/cmallparty/171220/contents.htm'

# Details


In [6]:
def getDetails(dfs,gid):
    df=dfs[0][:]
    df.set_index(0,inplace=True)
    df=df.T
    df['gid']=gid
    return df.reset_index(drop=True)

In [7]:
getDetails(dfs,gid)

,Title,Purpose,Category,gid
0,All-Party Parliamentary British-German Group,To promote mutual understanding and good relat...,Country Group,tmp


## Officers

In [8]:
def getOfficers(dfs, gid):
    df=dfs[1][:]
    df.columns=['Property','Name', 'Party']
    df.set_index('Property',inplace=True)
    df=df[2:]
    df['gid']=gid
    return df.reset_index(drop=True)

In [9]:
 getOfficers(dfs, gid)

,Name,Party,gid
0,Paul Farrelly,Labour,tmp
1,Lord Anderson of Swansea,Labour,tmp
2,Lord Dykes,Crossbench,tmp
3,Lord Balfe,Conservative,tmp
4,Lord Davies of Stamford,Labour,tmp
5,Mr Roger Godsiff,Labour,tmp
6,Mark Pritchard,Conservative,tmp
7,Mr Mark Hendrick,Labour (Co-op),tmp
8,David T. C. Davies,Conservative,tmp


## Contacts

In [10]:
def getContacts(dfs,gid):
    df=dfs[2][:]
    df.rename(columns=df[0], inplace=True)
    df['gid']=gid
    return df[1:].reset_index(drop=True)

In [11]:
getContacts(dfs,gid)

,Contact Details,gid
0,"Registered Contact: Paul Farrelly MP, House o...",tmp


## Annual General Meeting

In [12]:
def getAGM(dfs,gid):
    df=dfs[3].loc[:]
    df['gid']=gid
    return df.pivot(index='gid',columns=0,values=1).reset_index()

In [13]:
getAGM(dfs,gid)

,gid,Annual General Meeting,Date of most recent AGM in this Parliament,Did the group publish an income and expenditure statement relating to the AGM above?,Next reporting deadline,Reporting year
0,tmp,NaN,13/07/2016,No,02/10/2017,02 Jun to 01 Jun


## Registerable benefits received by the group

In [14]:
def getRegBenefits(dfs,gid):
    df=dfs[4][:]
    df.rename(columns=df.iloc[0], inplace=True)
    df['gid']=gid
    if len(df)>2:
        df.columns=['Source', 'Value', 'Received', 'Registered', 'gid']
    return df[3:]

In [15]:
getRegBenefits(dfs,gid)

,Registrable benefits received by the group,gid


## Benefits In Kind

In [16]:
def getInKindBenefits(dfs,gid):
    if len(dfs)>5:
        df=dfs[5][:]
        df.rename(columns=df.iloc[1], inplace=True)
        df=df[2:]
        df['gid']=gid
    else: return pd.DataFrame()
    return df

In [17]:
getInKindBenefits(dfs,gid)

,Source,Description,"Value £s In bands of £1,500",Received,Registered,gid
2,Rolls Royce Plc,Flights and accommodation for visit to Berlin ...,9001-10500,17/09/2016,27/10/2016,tmp


## Aggregator

In [18]:
df_details=pd.DataFrame() #getDetails(dfs,gid)
df_officers=pd.DataFrame() #getOfficers
df_contacts=pd.DataFrame() #getContacts(dfs,gid)
df_AGM=pd.DataFrame() #getAGM(dfs,gid)
df_regBenefits=pd.DataFrame() #getRegBenefits(dfs,gid)
df_inKindBenefits=pd.DataFrame() #getInKindBenefits(dfs,gid)

In [19]:
#doc = etree.HTML(urlopen(url).read())
#for x in doc.xpath('//*[@id="mainTextBlock"]/ul[1]/li/a'):
#    print(x)

In [20]:
#getDetails(dfs,gid)
#getOfficers(dfs,gid)
#getContacts(dfs,gid)
#getAGM(dfs,gid)
#getRegBenefits(dfs,gid)
#getInKindBenefits(dfs,gid)

In [21]:
doc = etree.HTML(requests.get("https://www.publications.parliament.uk/pa/cm/cmallparty/170502/contents.htm").text)
for x in doc.xpath('//*[@id="mainTextBlock"]/ul/li/a'):#doc.xpath('//p[@class="contentsLink"]/a'):
    print(x.text, x.attrib['href'])
    r = requests.get(urljoin('https://www.publications.parliament.uk/pa/cm/cmallparty/170502/', x.attrib['href']))
    gid= x.attrib['href'].replace('.htm','')
    dfs=pd.read_html(r.text)
    df_details=pd.concat([df_details,getDetails(dfs,gid)])
    df_officers=pd.concat([df_officers,getOfficers(dfs,gid)]) #getOfficers
    df_contacts=pd.concat([df_contacts,getContacts(dfs,gid)]) #getContacts(dfs,gid)
    df_AGM=pd.concat([df_AGM,getAGM(dfs,gid)]) #getAGM(dfs,gid)
    df_regBenefits=pd.concat([df_regBenefits,getRegBenefits(dfs,gid)]) #getRegBenefits(dfs,gid)
    df_inKindBenefits=pd.concat([df_inKindBenefits,getInKindBenefits(dfs,gid)])

Afghanistan afghanistan.htm
Africa africa.htm
African Great Lakes Region african-great-lakes-region.htm
Albania albania.htm
Algeria algeria.htm
Angola angola.htm
Argentina argentina.htm
Armenia armenia.htm
Australia and New Zealand australia-and-new-zealand.htm
Austria austria.htm
Azerbaijan azerbaijan.htm
Bahamas bahamas.htm
Bahrain bahrain.htm
Bangladesh bangladesh.htm
Belarus belarus.htm
Belize belize.htm
Bosnia and Herzegovina bosnia-and-herzegovina.htm
Botswana botswana.htm
Brazil brazil.htm
Bulgaria bulgaria.htm
Burma burma.htm
Cambodia cambodia.htm
Canada canada.htm
Caribbean caribbean.htm
Catalonia catalonia.htm
Cayman Islands cayman-islands.htm
Central American central-american.htm
Chagos chagos.htm
Channel Islands channel-islands.htm
Chile chile.htm
China china.htm
Chinese in Britain chinese-in-britain.htm
Colombia colombia.htm
Croatia croatia.htm
Cuba cuba.htm
Cyprus cyprus.htm
Czech and  Slovak czech-and-slovak.htm
Denmark denmark.htm
Egypt egypt.htm
Eritrea eritrea.htm
Est

Customer Service customer-service.htm
Cyber Security cyber-security.htm
Cycling cycling.htm
Dairy dairy.htm
Dalits dalits.htm
Dance dance.htm
Data Analytics data-analytics.htm
Deafness deafness.htm
Death Penalty death-penalty.htm
Debt and Personal Finance debt-and-personal-finance.htm
Defence and Security Issues defence-and-security-issues.htm
Dementia dementia.htm
Democracy and Human Rights in the Gulf democracy-and-human-rights-in-the-gulf.htm
Democratic Participation democratic-participation.htm
Dentistry and Oral Health dentistry-and-oral-health.htm
Design and Innovation design-and-innovation.htm
Diabetes diabetes.htm
Digital Crime digital-crime.htm
Digital Economy digital-economy.htm
Disability disability.htm
District Councils district-councils.htm
Dog Welfare dog-welfare.htm
Domestic Violence domestic-violence.htm
Drones drones.htm
Drug Policy Reform drug-policy-reform.htm
Dyslexia and Other Specific Learning Difficulties dyslexia-and-other-specific-learning-difficulties.htm
East

Muscular Dystrophy muscular-dystrophy.htm
Music music.htm
Music Education music-education.htm
Mutuals mutuals.htm
Myalgic Encephalomyelitis (ME) myalgic-encephalomyelitis-me.htm
National Parks national-parks.htm
Nuclear Energy nuclear-energy.htm
Nursery Schools and Nursery Classes nursery-schools-and-nursery-classes.htm
Obesity obesity.htm
Occupational Safety and Health occupational-safety-and-health.htm
Off-Patent Drugs off-patent-drugs.htm
Offshore Oil and Gas offshore-oil-and-gas.htm
Oil Refining Sector oil-refining-sector.htm
Opera opera.htm
Oral Hormone Pregnancy Tests oral-hormone-pregnancy-tests.htm
Osteoposis osteoposis.htm
Ovarian Cancer ovarian-cancer.htm
Packaging Manufacturing Industry packaging-manufacturing-industry.htm
Pakistan Minorities pakistan-minorities.htm
Pancreatic Cancer pancreatic-cancer.htm
Parents and Families parents-and-families.htm
Park Homes park-homes.htm
Parkinson's parkinsons.htm
Patient and Public Involvement in Health and Social Care patient-and-publ

In [22]:
df_details.to_csv('df_details.csv',index=False)
df_officers.to_csv('df_officers.csv',index=False)
df_contacts.to_csv('df_contacts.csv',index=False)
df_AGM.to_csv('df_AGM.csv',index=False)
df_regBenefits.to_csv('df_regBenefits.csv',index=False)
df_inKindBenefits.to_csv('df_inKindBenefits.csv',index=False)

## Example Queries

In [23]:
!wc -l df_*.csv

     632 df_AGM.csv
     632 df_contacts.csv
     632 df_details.csv
     389 df_inKindBenefits.csv
    4122 df_officers.csv
     348 df_regBenefits.csv
    6755 total


In [42]:
!ls -al *sqlite

-rw-r--r--@ 1 ajh59  1182653967   16809984  3 Jul 16:23 appg_cache.sqlite
-rw-r--r--@ 1 ajh59  1182653967  198070272 10 May 14:55 parlidata_cache.sqlite
-rw-r--r--@ 1 ajh59  1182653967          0  3 Jul 15:46 scraperwiki.sqlite
